In [2]:
import pandas as pd
import json
import nltk
import spacy
from nltk import word_tokenize
from spacy import displacy
nlp = spacy.load("en_core_web_sm")
nltk.download('universal_tagset')
pd.set_option("display.max_rows", None)

[nltk_data] Downloading package universal_tagset to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


This is a string from multiwoz.ipynb

In [3]:
docs = ["Boss is planning a vacation tomorrow night and really could use some help finding a good place to stay in town.",
        "I`m going to stay here 2 for 5 nights starting Sunday",
        "I need to arrive at 13:45",
        "I need the travel day to be Sunday",
        "I'll be going to Cambridge and arrive by 13:45", 
        "Train TR7468 will arrive at 13:23.",
        "Bears wake up at spring"]

In [4]:
tags = []
doc = nlp(docs[0])
for token in doc:
    tags.append([token.text, token.lemma_, token.ent_type_, token.pos_, token.tag_, token.dep_, token.head])
postok = {str(i): list(doc).index(i) for i in doc}
pd.DataFrame(tags, columns = ["Token", "lemma", "entity", "full", "entity", "dep type", "head"])

,Token,lemma,entity,full,entity,dep type,head
0,Boss,boss,,NOUN,NN,nsubj,planning
1,is,be,,AUX,VBZ,aux,planning
2,planning,plan,,VERB,VBG,ROOT,planning
3,a,a,,DET,DT,det,vacation
4,vacation,vacation,,NOUN,NN,dobj,planning
5,tomorrow,tomorrow,DATE,NOUN,NN,compound,night
6,night,night,,NOUN,NN,npadvmod,planning
7,and,and,,CCONJ,CC,cc,planning
8,really,really,,ADV,RB,advmod,use
9,could,could,,AUX,MD,aux,use


In [5]:
displacy.render(doc, jupyter=True)

Я хочу развернуть полное предложение - ивент

In [6]:
tok = doc[postok["tomorrow"]] #Time token
tok_seq = []
token_head = tok
while True:
    filtered_tok_seq = [] # filter extra dependencies of root word
    head_tok_seq = [*[i for i in token_head.lefts if i not in tok_seq], token_head, *[i for i in token_head.rights if i not in tok_seq]]
    for j in range(len(head_tok_seq)):
        if (head_tok_seq[j].dep_ in ["dobj", "nsubj", "ROOT"]) or (head_tok_seq[j] is token_head):
            filtered_tok_seq.append(head_tok_seq[j])
    tok_seq.extend(filtered_tok_seq)
    if (token_head.dep_ == "ROOT"):
        break
    token_head = token_head.head
tok_seq

[tomorrow, night, Boss, planning, vacation]

Получили из токена времени tomorrow полное предложение - Boss, planning, vacation, tomorrow

### Sentence with No

In [7]:
tags = []
doc = nlp("I will not arrive today at 5 a.m. ")
for token in doc:
    tags.append([token.text, token.lemma_, token.ent_type_, token.pos_, token.tag_, token.dep_, token.head])
postok = {str(i): list(doc).index(i) for i in doc}
pd.DataFrame(tags, columns = ["Token", "lemma", "entity", "full", "entity", "dep type", "head"])

,Token,lemma,entity,full,entity,dep type,head
0,I,I,,PRON,PRP,nsubj,arrive
1,will,will,,AUX,MD,aux,arrive
2,not,not,,PART,RB,neg,arrive
3,arrive,arrive,,VERB,VB,ROOT,arrive
4,today,today,DATE,NOUN,NN,npadvmod,arrive
5,at,at,,ADP,IN,prep,arrive
6,5,5,TIME,NUM,CD,pobj,at
7,a.m.,a.m.,TIME,NOUN,NN,npadvmod,arrive


In [8]:
displacy.render(doc, jupyter=True)

Возможно лучше нужно запоминать первоначальный ROOT, а потом от него смотреть - sub obj date и так далее

In [9]:
tok = doc[postok["nights"]] #Time token
tok_seq = []
token_head = tok
while True:
    filtered_tok_seq = [] # filter extra dependencies of root word
    head_tok_seq = [*[i for i in token_head.lefts if i not in tok_seq], token_head, *[i for i in token_head.rights if i not in tok_seq]]
    for j in range(len(head_tok_seq)):
        if (head_tok_seq[j].dep_ not in ["dobj", "nsubj", "nummod", "ROOT"]) or (head_tok_seq[j] is token_head):
            filtered_tok_seq.append(head_tok_seq[j])
    tok_seq.extend(filtered_tok_seq)
    if (token_head.dep_ == "ROOT"):
        break
    token_head = token_head.head
tok_seq

KeyError: 'nights'

Попробуем с событиями

In [ ]:
test_sents = ["The ship will arrive at 7 pm"]
roots_collection = {}
for sent in test_sents:
    tags = []
    doc = nlp(sent)
    for token in doc:
        tags.append({"text": token.text, "lemma": token.lemma_, "entity": token.ent_type_, "pos_tag": token.pos_,"tag": token.tag_,"dep_type": token.dep_,"head": token.head})
    postok = {str(i): list(doc).index(i) for i in doc}

    for i in range(len(tags)):
        tok = None
        if (tags[i]["entity"] in ["DATE", "TIME"]):
            tok = doc[i]
        else:
            continue
        tok_seq = []
        token_head = tok
        while True:
            filtered_tok_seq = [] # filter extra dependencies of root word
            head_tok_seq = [*[i for i in token_head.lefts if i not in tok_seq], token_head, *[i for i in token_head.rights if i not in tok_seq]]
            for j in range(len(head_tok_seq)):
                if (head_tok_seq[j].dep_ in ["dobj", "nsubj", "nummod", "ROOT", "neg"]) or (head_tok_seq[j] is token_head):
                    filtered_tok_seq.append(head_tok_seq[j])
            tok_seq.extend(filtered_tok_seq)
            if (token_head.dep_ == "ROOT"):
                break
            token_head = token_head.head
        print(f"# Extracted seq form time token [{tok}]: \n  {tok_seq} \n    Root: [{token_head}]")
        if (token_head not in roots_collection):
            roots_collection[token_head] = doc
            print(f"---- ROOT: [{token_head}] DOC: {doc} ----")

# Extracted seq form time token [7]: 
  [7, pm, at, ship, arrive] 
    Root: [arrive]
---- ROOT: [arrive] DOC: The ship will arrive at 7 pm ----
# Extracted seq form time token [pm]: 
  [7, pm, at, ship, arrive] 
    Root: [arrive]


Выделим этот код в функцию и насобираем подобные предложения по датасетам

## Word2vec

In [ ]:
from gensim.models import Word2Vec
from gensim.test.utils import common_texts
import gensim.downloader
model = Word2Vec(common_texts, vector_size=100, window=5, min_count=1, workers=4)
model.save("word2vec.model")

In [ ]:
glove_vectors = gensim.downloader.load('glove-twitter-25')

[==================================================] 100.0% 104.8/104.8MB downloaded


In [ ]:
glove_vectors.get_vector("plan")

array([ 0.47415 ,  0.93673 , -0.62139 , -0.013054,  0.98956 ,  0.45873 ,
       -0.02254 ,  0.6956  ,  0.44207 , -0.44703 ,  0.33811 , -0.92783 ,
       -3.6716  ,  0.84413 ,  1.1669  ,  0.50926 ,  0.73615 ,  0.10636 ,
       -0.24516 , -1.0525  ,  0.44059 , -0.86163 , -0.39066 , -0.075361,
        0.029276], dtype=float32)

In [ ]:
glove_vectors.most_similar("plan")

[('media', 0.8908759951591492),
 ('personal', 0.8748670816421509),
 ('control', 0.8535847663879395),
 ('idea', 0.8530606031417847),
 ('general', 0.852401614189148),
 ('debate', 0.8438900709152222),
 ('social', 0.842548131942749),
 ('sales', 0.8322349190711975),
 ('service', 0.8312507271766663),
 ('resume', 0.8249419927597046)]

Bad pipe message: %s [b'8?$\x8e\xde\xf3Z\t\x8c\xb6\xa9\x14\x93\xff\xd0\xf1Q\x94 \xca\x88\xfeK\xb8\xc6\xc7\xe8e\x9dI\xa7\xc4[\xc54\xb9\xcbG<\xf4\x17\x1f6c\xbfu\xd9\xf8\x88\x84\xdf\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00', b'\x1e\x00\x1c\x04\x03\x05\x03\x06\x03\x08\x07\x08']
Bad pipe message: %s [b'\t\x08\n\x08\x0b\x08\x04']
Bad pipe message: %s [b'\xd3u\x83\x94\xc4\x97\x8a\x08&\x8f\x84Z\x14=\xfdM\x0ct \xbbY\xf3\x0eB\xda\x976\xab\x11D\xe2\xc0\x0fY\xab\xd9\xd2a5:\x17\xa1\x94rL(\x93o\xfb\x84\x91\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00\x1c\x04\x03\x05\x03']
Bad pipe message: %s [b'\xe